In [15]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [18]:
import os
import openai
import PyPDF2
import random
import pinecone

from openai import OpenAI

In [6]:
os.environ["OPENAI_API_KEY"] = "XXXX"
client = OpenAI()
print(client.models.list())

SyncPage[Model](data=[Model(id='gpt-3.5-turbo-0613', created=1686587434, object='model', owned_by='openai'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-16k', created=1683758102, object='model', owned_by='openai-internal'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='system'), Model(id='tts-1-hd', created=1699046015, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='gpt-4-0125-preview', created=1706037612, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-3.5-turbo-instruct-0914', created=1694122472, object='model', owned_by='system'), Model(id='gpt-3.5-turbo', creat

In [21]:
def load_pdf(file_name):
    pdf_file = open(file_name,'rb')
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    text_content=""
    for page in range(len(pdf_reader.pages)):
        text_content += pdf_reader.pages[page].extract_text()
    pdf_file.close()
    return text_content

In [9]:
def chunk_text(text,chunk_size=1500,chunk_overlap=100,by='word'):
    if by not in ['word','char']:
        raise ValueError("Please Insert correct 'by' input, should be word or char")
    chunks = []
    if by == 'word':
        text = text.split()
    elif by == 'char':
        text = text
    current_chunk_start = 0
    while current_chunk_start < len(text):
        current_chunk_end = current_chunk_start +  chunk_size
        if by == 'word':
            chunk = " ".join(text[current_chunk_start:current_chunk_end])
        else:
            chunk = text[current_chunk_start:current_chunk_end]

        chunks.append(chunk)
        current_chunk_start += (chunk_size - chunk_overlap)


    return chunks

In [22]:
pdf_loaded = load_pdf("C:/Users/prohr/Downloads/state_of_ai_docs.pdf")

In [23]:
chunks = chunk_text(pdf_loaded, by='char')

In [25]:
len(chunks)

28

In [27]:
for i in range(len(chunks)):
    print(chunks[i])
    break

As organizations rapidly deploy generative AI tools, survey respondents 
expect significant effects on their industries and workforces.The state of AI in 
2023: Generative AI’s 
breakout year
August 2023The state of AI in 2023: Generative AI’s breakout yearThe latest annual McKinsey Global Survey  on the current  
state of AI confirms the explosive growth of generative AI  
(gen AI) tools. Less than a year after many of these tools debuted, 
one-third of our survey respondents say their organizations are 
using gen AI regularly in at least one business function. Amid 
recent advances, AI has risen from a topic relegated to tech 
employees to a focus of company leaders: nearly one-quarter  
of surveyed C-suite executives say they are personally using  
gen AI tools for work, and more than one-quarter of respondents 
from companies using AI say gen AI is already on their boards’ 
agendas. What’s more, 40 percent of respondents say their 
organizations will increase their investment in AI

In [28]:
from pinecone import Pinecone

pc = Pinecone(api_key="ABCD")
index = pc.Index("Txt")

In [30]:
for i in range(len(chunks)):
    vec = client.embeddings.create(
        model= "text-embedding-ada-002",
        input= chunks[i]
    )
    vec = vec.data[0].embedding

In [32]:
upsert_response = index.upsert(
    vectors = [
     (
         str(i),
         vec,
         {"chunk_content" : chunks[i]}
     )   
    ])

In [34]:
user_request = input("Ask some question regarding the document: ")

user_vector = client.embeddings.create(
    model="text-embedding-ada-002",
    input=user_request)

user_vector = user_vector.data[0].embedding

Ask some question regarding the document:  Recent trend in AI


In [40]:
matches = index.query(
    vector=user_vector,
    top_k=1,
    include_metadata=True,
    )

messages = [{"role" : "system", "content":"""I want you to act as a support agent. Your name is "My Super Assistant". You will provide me with answers from the given info. If the answer is not included, say exactly "Ooops! I don't know that." and stop after that. Refuse to answer any question not about the info. Never break character and always answer on the given text."""}]
messages.append({"role":"user","content": matches['matches'][0]['metadata']['chunk_content']})
messages.append({"role":"user","content":user_request})

print(messages)

[{'role': 'system', 'content': 'I want you to act as a support agent. Your name is "My Super Assistant". You will provide me with answers from the given info. If the answer is not included, say exactly "Ooops! I don\'t know that." and stop after that. Refuse to answer any question not about the info. Never break character and always answer on the given text.'}, {'role': 'user', 'content': 'nd \ncapabilities that allow them to generate value. One way to interpret this is that “the rich \nare getting richer” when it comes to extracting value from AI. We’ll be interested in seeing \nwhether the great interest in generative AI opens the door to higher overall adoption of AI \ngoing forward.\n21\nThe state of AI in 2023: Generative AI’s breakout yearQuantumBlack AI, by McKinsey  \nAugust 2023  \nCopyright © McKinsey & Company  \nDesigned by McKinsey Global Publishing\nMcKinsey.com\n @McKinsey  \n  @McKinsey\nScan • Download • Personalize\nFind more content like this on the \nMcKinsey Insigh

In [45]:
chat_response  = client.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages=messages,
    temperature=0,
    max_tokens=400 
)

messages.append({"role":"assistant","content":chat_response.choices[0].message.content})
print("Assistant:", chat_response.choices[0].message.content)
print()
print("Context: ", matches['matches'][0]['metadata']['chunk_content'])
print("$$$$$")

Assistant: The recent trend in AI is the rise of Generative AI, which is expected to have a breakout year in 2023 according to QuantumBlack AI by McKinsey.

Context:  nd 
capabilities that allow them to generate value. One way to interpret this is that “the rich 
are getting richer” when it comes to extracting value from AI. We’ll be interested in seeing 
whether the great interest in generative AI opens the door to higher overall adoption of AI 
going forward.
21
The state of AI in 2023: Generative AI’s breakout yearQuantumBlack AI, by McKinsey  
August 2023  
Copyright © McKinsey & Company  
Designed by McKinsey Global Publishing
McKinsey.com
 @McKinsey  
  @McKinsey
Scan • Download • Personalize
Find more content like this on the 
McKinsey Insights App
$$$$$
